In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import  ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
train = pd.read_csv(r"C:\Workspace\Pri\data science and ML practice\jupyter\DataBases\kaggle_dataset\train.csv")
test = pd.read_csv(r"C:\Workspace\Pri\data science and ML practice\jupyter\DataBases\kaggle_dataset\test.csv")


print("shape of train df =",train.shape)

print("shape of test df =",test.shape)

shape of train df = (1460, 81)
shape of test df = (1459, 80)


In [5]:
#   TRAIN DATASET : WE ARE GOING TO DIVIDE DATASET INTO X AND Y 
#                 : X_TRAIN  AND  y_TRAIN 
#                 : X_TRAIN = Independent variables / Identifiers (Matrix)
#                 : y_TRAIN = Dependent variable / Measurement (Vector)

X_train = train.drop(columns="SalePrice")
y_train = train["SalePrice"]

X_test = test.copy()

print("shape of X_train df =", X_train.shape) # (rows , colums)
print("shape of y_train df =", y_train.shape)  # ( only rows of one colums)  [ so that resone colum side blank]

print("shape of X_test df =", X_test.shape)

shape of X_train df = (1460, 80)
shape of y_train df = (1460,)
shape of X_test df = (1459, 80)


# Missing Value Imputation

In [8]:
isnull_sum = X_train.isnull().sum()
isnull_sum

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 80, dtype: int64

In [7]:
num_train = X_train.select_dtypes(include=["int64","float64"]).columns
num_train

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')

In [11]:
num_train_miss = [var for var in  num_train if isnull_sum[var]>0]
num_train_miss

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [12]:
cat_train = X_train.select_dtypes(include=["O"]).columns  # O = Object
cat_train

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [13]:
cat_train_miss = [var for var in  cat_train if isnull_sum[var]>0]
cat_train_miss

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

# we are using different Strategy for Variables

In [14]:
num_train_mean = ['LotFrontage']                              # ---------MEAN

num_train_median =['MasVnrArea', 'GarageYrBlt']                #---------MEDIAN

cat_train_mode = ['Alley',                                     #---------MODE ( MOST FREQUENT )
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu']

cat_train_constant = ['GarageType',                           # ----------CONSTANT
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

# For that we are creating PIPELINE

In [17]:
num_train_mean_imputer = Pipeline(steps = [("imputer", SimpleImputer(strategy = "mean"))])

num_train_median_imputer = Pipeline(steps = [("imputer", SimpleImputer(strategy = "median"))])

cat_train_mode_imputer = Pipeline(steps = [("imputer", SimpleImputer(strategy = "most_frequent"))])

cat_train_constant_imputer = Pipeline(steps = [("imputer", SimpleImputer(strategy = "constant", fill_value="Missing"))])

In [18]:
preprocessor = ColumnTransformer( transformers=[("mean_imputer", num_train_mean_imputer, num_train_mean ) ,
                                                ("median_imputer", num_train_median_imputer, num_train_median ) ,
                                                ("mode_imputer", cat_train_mode_imputer, cat_train_mode ) ,
                                                ("constant_imputer", cat_train_constant_imputer, cat_train_constant ) ,])

In [19]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical',
                                  'FireplaceQu']),
                                ('constant_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='Missing',
                                                                strategy='constant'))]),
                                 ['GarageType', 'GarageFinish', 'GarageQual',
                                  'GarageCond', 'PoolQC', 'Fence',
                                  'MiscFeature'])])

In [23]:
preprocessor.named_transformers_["mean_imputer"].named_steps["imputer"].statistics_

array([70.04995837])

In [24]:
train["LotFrontage"].mean()

70.04995836802665

In [25]:
preprocessor.named_transformers_["median_imputer"].named_steps["imputer"].statistics_

array([   0., 1980.])

In [33]:
train['MasVnrArea'].median()

0.0

In [34]:
train['GarageYrBlt'].median()

1980.0

In [35]:
preprocessor.named_transformers_["mode_imputer"].named_steps["imputer"].statistics_

array(['Grvl', 'None', 'TA', 'TA', 'No', 'Unf', 'Unf', 'SBrkr', 'Gd'],
      dtype=object)

In [36]:
preprocessor.named_transformers_["constant_imputer"].named_steps["imputer"].statistics_

array(['Missing', 'Missing', 'Missing', 'Missing', 'Missing', 'Missing',
       'Missing'], dtype=object)

In [37]:
X_train_clean = preprocessor.transform(X_train)
X_test_clean = preprocessor.transform(X_test)

In [38]:
X_train_clean

array([[65.0, 196.0, 2003.0, ..., 'Missing', 'Missing', 'Missing'],
       [80.0, 0.0, 1976.0, ..., 'Missing', 'Missing', 'Missing'],
       [68.0, 162.0, 2001.0, ..., 'Missing', 'Missing', 'Missing'],
       ...,
       [66.0, 0.0, 1941.0, ..., 'Missing', 'GdPrv', 'Shed'],
       [68.0, 0.0, 1950.0, ..., 'Missing', 'Missing', 'Missing'],
       [75.0, 0.0, 1965.0, ..., 'Missing', 'Missing', 'Missing']],
      dtype=object)

In [39]:
X_test_clean

array([[80.0, 0.0, 1961.0, ..., 'Missing', 'MnPrv', 'Missing'],
       [81.0, 108.0, 1958.0, ..., 'Missing', 'Missing', 'Gar2'],
       [74.0, 0.0, 1997.0, ..., 'Missing', 'MnPrv', 'Missing'],
       ...,
       [160.0, 0.0, 1960.0, ..., 'Missing', 'Missing', 'Missing'],
       [62.0, 0.0, 1980.0, ..., 'Missing', 'MnPrv', 'Shed'],
       [74.0, 94.0, 1993.0, ..., 'Missing', 'Missing', 'Missing']],
      dtype=object)

In [40]:
preprocessor.transformers_

[('mean_imputer',
  Pipeline(steps=[('imputer', SimpleImputer())]),
  ['LotFrontage']),
 ('median_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))]),
  ['MasVnrArea', 'GarageYrBlt']),
 ('mode_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))]),
  ['Alley',
   'MasVnrType',
   'BsmtQual',
   'BsmtCond',
   'BsmtExposure',
   'BsmtFinType1',
   'BsmtFinType2',
   'Electrical',
   'FireplaceQu']),
 ('constant_imputer',
  Pipeline(steps=[('imputer',
                   SimpleImputer(fill_value='Missing', strategy='constant'))]),
  ['GarageType',
   'GarageFinish',
   'GarageQual',
   'GarageCond',
   'PoolQC',
   'Fence',
   'MiscFeature']),
 ('remainder',
  'drop',
  [0,
   1,
   2,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   27,
   28,
   29,
   34,
   36,
   37,
   38,
   39,
   40,
   41,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
  

In [41]:
X_train_clean_df = pd.DataFrame(X_train_clean, columns=num_train_mean+num_train_median+cat_train_mode+cat_train_constant)

In [42]:
X_train_clean_df.head()

,LotFrontage,MasVnrArea,GarageYrBlt,Alley,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,65.0,196.0,2003.0,Grvl,BrkFace,Gd,TA,No,GLQ,Unf,SBrkr,Gd,Attchd,RFn,TA,TA,Missing,Missing,Missing
1,80.0,0.0,1976.0,Grvl,None,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing
2,68.0,162.0,2001.0,Grvl,BrkFace,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing
3,60.0,0.0,1998.0,Grvl,None,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,Unf,TA,TA,Missing,Missing,Missing
4,84.0,350.0,2000.0,Grvl,BrkFace,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing


In [43]:
X_train_clean_df.isnull().sum()

LotFrontage     0
MasVnrArea      0
GarageYrBlt     0
Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
Electrical      0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

In [44]:
train["Alley"].value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

In [45]:
X_train_clean_df["Alley"].value_counts()

Grvl    1419
Pave      41
Name: Alley, dtype: int64

In [48]:
train['MiscFeature'].value_counts()

Shed    49
Gar2     2
Othr     2
TenC     1
Name: MiscFeature, dtype: int64

In [47]:
X_train_clean_df['MiscFeature'].value_counts()

Missing    1406
Shed         49
Gar2          2
Othr          2
TenC          1
Name: MiscFeature, dtype: int64